In [ ]:
import os
import numpy as np
import pandas as pd
import qtconsole
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import BeamDynamics as bd
import copy
try:
    import ROOT
except:
    print('Root framework not available.')

In [ ]:
from importlib import reload
reload(bd)

In [ ]:
%qtconsole

In [ ]:
%connect_info

In [ ]:
%matplotlib inline
# %matplotlib notebook

# FODO

## Analytical relations from thin lens approximation

### General relations

In [ ]:
def fodo_thin_lcell(betaPlus, psi):
    Lcell = betaPlus * np.sin(psi) / (1 + np.sin(psi/2.))
    return Lcell

def fodo_thin_betaminus(Lcell, psi):
    betaMinus = Lcell * (1 - np.sin(psi/2.)) / np.sin(psi)
    return betaMinus

def fodo_thin_f(Lcell, psi):
    f = Lcell / (4 * np.sin(psi/2.))
    return f

def fodo_thin_lquad_1(f, kQuad):
    Lquad = 1 / (kQuad * f)
    return Lquad

def fodo_thin_ldrift_1(Lcell, Lquad):
    Ldrift = (Lcell - 2.*Lquad) / 2.
    return Ldrift

def fodo_thin_lquad_2(Lcell, Ldrift):
    Lquad = (Lcell - 2.*Ldrift) / 2.
    return Lquad

### Assumption: BetaMax determined by emittance and aperture

In [ ]:
def fodo_thin_betaplus(Ra, Fa, gammaRel, emitn):
    betaRel = bd.gamma_to_beta(gammaRel)
    betaPlus = (Ra/Fa)**2. * betaRel * gammaRel / emitn
    return betaPlus

### Assumption: Using the max. available quad gradient

In [ ]:
def fodo_thin_ldrift_2(Ra, Fa, gammaRel, emitn, psi, Gquad):
    betaPlus = fodo_thin_betaplus(Ra, Fa, gammaRel, emitn)
    term1 = betaPlus * np.sin(psi) / (2.*(1.+np.sin(psi/2.)))
    kQuad = bd.quad_strength(Gquad, bd.gamma_to_p(gammaRel, -11))
    term2 = 4. * np.sin(psi/2.) * (1.+np.sin(psi/2.)) / (betaPlus * np.sin(psi) * kQuad)
    Ldrift = term1 - term2
    return Ldrift

### Assumption: Limit aperture to length ratio of the quad

In [ ]:
def fodo_thin_lquad_3(f, quadGradMax, p, Rpole, RpoleToLquadMaxRatio):
    kQuadMax = bd.quad_strength(quadGradMax, p)
    LquadMin = 1 / (kQuadMax * f)
    takeLquadMinInds = Rpole / LquadMin < RpoleToLquadMaxRatio
    Lquad = np.zeros(takeLquadMinInds.shape)
    Lquad[takeLquadMinInds] = LquadMin[takeLquadMinInds]
    Lquad[~takeLquadMinInds] = Rpole / RpoleToLquadMaxRatio
    return Lquad

## Results from Elegant optimizations (thick lenses)

In [ ]:
data = {
    'Ra': [
             20.,      20.,      20.,      20.,      20.,      20.,      20.,
             25.,      30.,      35.,      40.,      45.,
             30.,
    ],   # [mm]
    'Fa': [
              3.,       3.,       3.,       3.,       3.,       3.,       3.,
              4.,       4.,       4.,       4.,       4.,
              4.,
    ],
    'psi': [
            22.5,      45.,      60.,      70.,   76.345,      90.,     100.,
          76.345,   76.345,   76.345,   76.345,   76.345,
             60.,
    ],   # [deg]
    'LquadHalf' : [
        0.107503, 0.112642, 0.129234, 0.144463, 0.156520, 0.193600, 0.256245,
        0.203432, 0.135236, 0.079521, 0.059732, 0.046750,
        0.000000,
    ],   # [m]
    'Ldrift': [
        0.070826, 0.235100, 0.266819, 0.259910, 0.243498, 0.169696, 0.034461,
        0.094104, 0.408817, 0.759710, 1.071240, 1.407448,
        0.000000,
    ],   # [m]
}
fodoDf = pd.DataFrame(data=data)
fodoDf['Lcell'] = 2.*2.*fodoDf['LquadHalf'] + 2.*fodoDf['Ldrift']

## Cell length vs phase advance

### Fix parameters

In [ ]:
Ra = 20.   # [mm]
Fa = 3.
p = 200.   # [MeV/c]
gammaRel = bd.p_to_gamma(p, -11)
emitn = 10e3   # [pi mm mrad]
betaPlus = fodo_thin_betaplus(Ra, Fa, gammaRel, emitn)
Gquad = 0.6 / 0.1   # [T/m]
kQuad = bd.quad_strength(Gquad, p)
print('gammaRel = {:.1f}.'.format(gammaRel))
print('betaPlus = {:.3f} m.'.format(betaPlus))
print('kQuad = {:.3f} 1/m2.'.format(kQuad))

### Select Elegant results

In [ ]:
fodoSel = fodoDf[fodoDf['Fa']==Fa]

### Variable parameter: Phase advance psi

In [ ]:
psi = np.linspace(fodoSel['psi'].min(), fodoSel['psi'].max())

### Compute curves from thin lens approx.

In [ ]:
LcellThin = fodo_thin_lcell(betaPlus, psi/180.*np.pi)
LquadThin = fodo_thin_lquad_1(fodo_thin_f(LcellThin, psi/180.*np.pi), kQuad)
LdriftThin = fodo_thin_ldrift_1(LcellThin, LquadThin)

### Compare thick lenses (Elegant) vs. thin lens approx.

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8,12))
ax[0].plot(fodoSel['psi'], fodoSel['Lcell'], 'o-')
ax[0].plot(psi, LcellThin, '--')
ax[0].grid()
ax[0].set_ylim((0, 1.2))
ax[0].set_xlabel('Phase advance [deg]')
ax[0].set_ylabel('FODO cell length [m]')
ax[0].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=8)
ax[1].plot(fodoSel['psi'], fodoSel['Ldrift'], 'o-')
ax[1].plot(psi, LdriftThin, '--')
ax[1].grid()
ax[1].set_ylim((0, 0.3))
ax[1].set_xlabel('Phase advance [deg]')
ax[1].set_ylabel('Drift length [m]')
ax[1].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=8)
ax[2].plot(fodoSel['psi'], fodoSel['LquadHalf']*2., 'o-')
ax[2].plot(psi, LquadThin, '--')
ax[2].grid()
ax[2].set_ylim((0, 0.6))
ax[2].set_xlabel('Phase advance [deg]')
ax[2].set_ylabel('Quad length [m]')
_ = ax[2].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=8)

## Cell length vs. aperture

### Fix parameters

In [ ]:
Fa = 4.
p = 200.   # [MeV/c]
gammaRel = bd.p_to_gamma(p, -11)
emitn = 10e3   # [pi mm mrad]
Gquad = 0.6 / 0.1   # [T/m]
kQuad = bd.quad_strength(Gquad, p)
psi = 76.345 / 180. * np.pi   # [rad]
print('gammaRel = {:.1f}.'.format(gammaRel))
print('kQuad = {:.3f} 1/m2.'.format(kQuad))

### Select Elegant results

In [ ]:
fodoSel = fodoDf[fodoDf['Fa']==Fa]

### Variable parameter

In [ ]:
Ra = np.linspace(20., fodoSel['Ra'].max())

### Compute curves from thin lens approx.

In [ ]:
LdriftThin = fodo_thin_ldrift_2(Ra, Fa, gammaRel, emitn, psi, Gquad)
LcellThin = fodo_thin_lcell(fodo_thin_betaplus(Ra, Fa, gammaRel, emitn), psi)
LquadThin = fodo_thin_lquad_2(LcellThin, LdriftThin)

### Compare thick lenses (Elegant) vs. thin lens approx.

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8,12))
ax[0].plot(fodoSel['Ra'], fodoSel['Lcell'], 'o-')
ax[0].plot(Ra, LcellThin, '--')
ax[0].grid()
ax[0].set_ylim((0, 3.5))
ax[0].set_xlabel('Aperture (iris radius) [mm]')
ax[0].set_ylabel('FODO cell length [m]')
ax[0].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=9)
ax[1].plot(fodoSel['Ra'], fodoSel['Ldrift'], 'o-')
ax[1].plot(Ra, LdriftThin, '--')
ax[1].grid()
ax[1].set_ylim((-0.2, 1.6))
ax[1].set_xlabel('Aperture (iris radius) [mm]')
ax[1].set_ylabel('Drift length [m]')
ax[1].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=9)
ax[2].plot(fodoSel['Ra'], fodoSel['LquadHalf']*2., 'o-')
ax[2].plot(Ra, LquadThin, '--')
ax[2].grid()
ax[2].set_ylim((0, 0.5))
ax[2].set_xlabel('Aperture (iris radius) [mm]')
ax[2].set_ylabel('Quad length [m]')
_ = ax[2].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=9)

## Maximize drift length

### Fix parameters

In [ ]:
Fa = 4.
emitn = 10e3   # [pi mm mrad]
Gquad = 1.0 / 0.1   # [T/m]

### Discrete parameters

In [ ]:
pArr = np.array((200., 350., 500.))   # [MeV/c]
gammaRelArr = bd.p_to_gamma(pArr, -11)
kQuadArr = bd.quad_strength(Gquad, pArr)

### Variable parameters

In [ ]:
Ra = np.linspace(20., 40.)
psi = np.linspace(10., 90.) / 180. * np.pi   # [rad]
Ra, psi = np.meshgrid(Ra, psi)
psiDeg = psi / np.pi * 180

In [ ]:
fig, ax = plt.subplots(figsize=(9,16), subplot_kw={"projection": "3d"})
for p, gammaRel, kQuad in zip(pArr, gammaRelArr, kQuadArr):
    # Compute curves from thin lens approx.
    LdriftThin = fodo_thin_ldrift_2(Ra, Fa, gammaRel, emitn, psi, Gquad)
    LcellThin = fodo_thin_lcell(fodo_thin_betaplus(Ra, Fa, gammaRel, emitn), psi)
    LquadThin = fodo_thin_lquad_2(LcellThin, LdriftThin)
    # Plot
    surf = ax.plot_surface(
        Ra, psiDeg, LdriftThin,
        cmap=cm.coolwarm, vmin=0, vmax=2.7,
        linewidth=0, rstride=5, cstride=5, antialiased=False
    )
fig.colorbar(surf, shrink=0.3, aspect=15)
ax.set_xlim(np.min(Ra), np.max(Ra))
ax.set_ylim(np.min(psiDeg), np.max(psiDeg))
ax.set_xlabel('Ra [mm]')
ax.set_ylabel('psi [deg]')
ax.set_zlabel('Ldrift [m]')
ax.view_init(elev=35., azim=20.)
ax.text(32.5, 80., 0, 'p = 200 MeV/c', (1,0,0.05), zorder=4)
ax.text(32.5, 80., 0.9, 'p = 350 MeV/c', (1,0,0.09), zorder=5)
_ = ax.text(37.5, 85., 2.5, 'p = 500 MeV/c', (1,0,0.11), zorder=6)

## Select working point

### Fix parameters

In [ ]:
Ra = 30.   # [mm]
Fa = 4.
p = 500.   # [MeV/c]
gammaRel = bd.p_to_gamma(p, -11)
emitn = 10e3   # [pi mm mrad]
betaPlus = fodo_thin_betaplus(Ra, Fa, gammaRel, emitn)
Rpole = 0.1   # [m]
Gquad = 1. / Rpole   # [T/m]
RpoleToLquadMaxRatio = 0.1
print('gammaRel = {:.1f}.'.format(gammaRel))
print('betaPlus = {:.3f} m.'.format(betaPlus))

### Select Elegant results

In [ ]:
fodoSel = fodoDf.iloc[-1,:]

### Variable parameter: Phase advance psi

In [ ]:
psiDeg = np.linspace(10., 90.)   # [deg]
psi = psiDeg / 180. * np.pi   # [rad]

### Compute curves from thin lens approx.

In [ ]:
LcellThin = fodo_thin_lcell(betaPlus, psi)
LquadThin = fodo_thin_lquad_1(fodo_thin_f(LcellThin, psi), kQuad)
LquadRealMagnet = fodo_thin_lquad_3(fodo_thin_f(LcellThin, psi), Gquad, p, Rpole, RpoleToLquadMaxRatio)
LdriftThin = fodo_thin_ldrift_1(LcellThin, LquadThin)
LdriftRealMagnet = fodo_thin_ldrift_1(LcellThin, LquadRealMagnet)

### Compare thick lenses (Elegant) vs. thin lens approx.

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8,12))
ax[0].plot(fodoSel['psi'], fodoSel['Lcell'], 'o-')
ax[0].plot(psiDeg, LcellThin, '--')
ax[0].grid()
ax[0].set_ylim((0, 3.5))
ax[0].set_xlabel('Phase advance [deg]')
ax[0].set_ylabel('FODO cell length [m]')
ax[0].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=8)
ax[1].plot(fodoSel['psi'], fodoSel['Ldrift'], 'o-')
ax[1].plot(psiDeg, LdriftThin, '--')
ax[1].plot(psiDeg, LdriftRealMagnet, '--')
ax[1].grid()
ax[1].set_ylim((0, 1.6))
ax[1].set_xlabel('Phase advance [deg]')
ax[1].set_ylabel('Drift length [m]')
ax[1].legend(('Elegant optim.', 'Analytical (thin lens)', 'Analytical (limit Rpole/Lquad ratio)'), loc=8)
ax[2].plot(fodoSel['psi'], fodoSel['LquadHalf']*2., 'o-')
ax[2].plot(psiDeg, LquadThin, '--')
ax[2].plot(psiDeg, LquadRealMagnet, '--')
ax[2].grid()
ax[2].set_ylim((0, 1.1))
ax[2].set_xlabel('Phase advance [deg]')
ax[2].set_ylabel('Quad length [m]')
_ = ax[2].legend(('Elegant optim.', 'Analytical (thin lens)', 'Analytical (limit Rpole/Lquad ratio)'), loc=8)

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some error.
</div>

<div class="alert alert-block alert-success">
Something good.
</div>